In [1]:
import pandas as pd
import numpy as np
import gradio as gr
import nltk
import spacy
import pickle
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nlp = spacy.load("en_core_web_sm")
nltk.download('vader_lexicon')

from absa_functions import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Evan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
def parse_targets(nlp, review):
    doc = nlp(review)
    targets = []
    target = ''

    for token in doc:
        if (token.dep_ in ['nsubj','dobj', 'pobj', 'ROOT']) and (token.pos_ in ['NOUN', 'PROPN', 'PRON']):
            target = token.text
            targets.append(target)

    return targets

In [3]:
def parse_adjectives(nlp, review):
    doc = nlp(review)
    adjectives = []
    adjective = ''

    for token in doc:
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            adjective = prepend + token.text
            adjectives.append(adjective)

    return adjectives

In [4]:
def get_topic_from_word(word, lda_model, topic_map):
    try:
        topics_raw = lda_model.get_term_topics(word, minimum_probability=0.0000001)
        topic_dict = {topic_map[tup[0]]: tup[1] for tup in topics_raw}
        best_topic = max(topic_dict, key=topic_dict.get)
    except:
        best_topic = 'miscellaneous'

    return best_topic

In [5]:
corpus = pickle.load(open('corpus.pkl', 'rb'))
dictionary = corpora.Dictionary.load('dictionary.gensim')
lda_model = LdaModel.load('best_lda_model.gensim')

In [6]:
sid = SentimentIntensityAnalyzer()

In [7]:
#numerical mapping for topics in LDA model
topic_map = {0: 'menu', 1: 'service', 2: 'miscellaneous', 3: 'place', 4: 'price', 5: 'food', 6: 'staff'}

In [9]:
review = 'increased amounts of kimchi in your diet leads to increased abundance of lactobacillus in your gut.'

In [10]:
count = 0
doc = nlp(review)
for token in doc:
    count += 1
    if count <= 20:
    # if 12 <= count <= 18:
        print(token.text, token.dep_, token.head.text, token.head.pos_, token.pos_,[child for child in token.children])

increased amod amounts NOUN VERB []
amounts nsubj leads VERB NOUN [increased, of, in]
of prep amounts NOUN ADP [kimchi]
kimchi pobj of ADP NOUN []
in prep amounts NOUN ADP [diet]
your poss diet NOUN PRON []
diet pobj in ADP NOUN [your]
leads ROOT leads VERB VERB [amounts, to, in, .]
to prep leads VERB ADP [abundance]
increased amod abundance NOUN VERB []
abundance pobj to ADP NOUN [increased, of]
of prep abundance NOUN ADP [lactobacillus]
lactobacillus pobj of ADP NOUN []
in prep leads VERB ADP [gut]
your poss gut NOUN PRON []
gut pobj in ADP NOUN [your]
. punct leads VERB PUNCT []


In [11]:
parse_targets(nlp, review)

['music']

In [12]:
parse_adjectives(nlp, review)

['Wonderful', 'beautiful', 'great']

In [13]:
get_topic_from_word(prepare_text_for_lda('entitlement')[0], lda_model, topic_map)

'miscellaneous'

In [22]:
def pos_prediction(restaurant_review):
    nlp = spacy.load("en_core_web_sm")
    targets = parse_targets(nlp, restaurant_review)
    adjectives = parse_adjectives(nlp, restaurant_review)

    outputs = []
    if len(targets) == len(adjectives): 
        for i in range(0, len(targets)):
            output = {}
            
            output.update({'aspect': targets[i], 'adjective': adjectives[i]})
            try:
                topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, topic_map)
            except:
                topic = 'miscellaneous'
            score = sid.polarity_scores(adjectives[i])['compound']
            sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
            output.update({'topic': topic, 'polarity': sentiment})
            outputs.append(output)
    elif len(targets) > len(adjectives):
        for i in range(0, len(targets)):
            output = {}
            try:
                topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, topic_map)
                score = sid.polarity_scores(adjectives[i])['compound']
                sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
                output.update({'aspect': targets[i], 'adjective': adjectives[i], 'topic': topic, 'polarity': sentiment})
            except IndexError:
                output.update({'aspect': targets[i], 'adjective': 'None', 'topic': targets[i], 'polarity': 'None'})
            
            outputs.append(output)
    elif len(targets) < len(adjectives):
        for i in range(0, len(adjectives)):
            output = {}
            try:
                topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, topic_map)
                score = sid.polarity_scores(adjectives[i])['compound']
                sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
                output.update({'aspect': targets[i], 'adjective': adjectives[i], 'topic': topic, 'polarity': sentiment})
            except IndexError:
                score = sid.polarity_scores(adjectives[i])['compound']
                sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
                output.update({'aspect': 'None', 'adjective': adjectives[i], 'topic': 'miscellaneous', 'polarity': sentiment})
            outputs.append(output)
    
    df = pd.DataFrame(outputs)
    boolean_series = ~df.aspect.isin(list(set(nltk.corpus.stopwords.words('english'))) + ['I'])
    output_df = df[boolean_series]
    return output_df
        


In [23]:
sample = '''I expected more. And maybe this is where Google reviews fall short. 
I wonder how many people have actually been to Spain and experienced real paella. 
This one was as decent as it could be but scallops were not juicy, the rice was quite dry, and seafood just not as fresh as you’d want. 
I appreciate the effort and the ambiance created at the restaurant. 
I can say good thing about the pimiento stuffed with beef cheeks — very tender and soft, as they should be. 
Then shrimps pil pil style definitely missed that strong garlic flavour and salt. 
Lovely service that made the stay very enjoyable'''

In [24]:
pos_prediction(sample)

,aspect,adjective,topic,polarity
1,reviews,short,price,neutral
3,people,real,place,neutral
4,Spain,as decent,miscellaneous,neutral
5,paella,juicy,staff,neutral
6,one,quite dry,price,neutral
8,scallops,good,menu,positive
9,rice,very tender,place,neutral
12,effort,garlic,menu,neutral
13,restaurant,Lovely,place,positive
15,thing,None,thing,None


In [30]:
iface = gr.Interface(fn=pos_prediction, 
                    inputs=gr.inputs.Textbox(lines=2, placeholder='Enter restaurant review here...'),
                    outputs=gr.outputs.Dataframe(headers=['Aspect','Adjective', 'Topic', 'Polarity']),
                    examples=[
                        ['The restaurant is too dark, and the bathroom was not clean. Also, everyone there is rude.'],
                        ['Fabulous dinner & environment but the older waiters have a real sense of entitlement.'],
                        ['Entrees were way too expensive.'],
                        ['The dinner was great, and the waiter was super friendly.']
                    ])
iface.launch(share=True)

Running locally at: http://127.0.0.1:7861/
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://58123.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://58123.gradio.app')

[2021-10-06 21:59:26,341] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "<ipython-input-28-2b8dce64a55e>", line 24, in pos_prediction
    topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, topic_map)
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\nlpenv\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\envs\nlpenv\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\envs\nlpenv\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\ProgramData\Anaconda3\envs\nlpenv\lib\site-packages\flask\app.py", line 1821, in handle_user_

In [ ]:
['topic', 'meta', 'polarity']

#group by topics, meta: ['delicious-beautiful', 'experience-outstanding']
#hide rows that are None
